# Notes for the Project Report

Initially started with LR, DT, RF and SVM.

Default model settings were used to see the performance before starting tuning. Accuracy appeared to be over 90%.

For hyperparameter tuning the Random Search approach was used as it is considered to work faster than Grid Search with relatively the same accuracy.

First iteration of tuning: all models have relatively the same accuracy ~96%, SVC model is time extensive so won't be used on the second iteration.

Second iteration: new sets of hyperparameters used for each model.

# LOADING TRAIN AND TEST DATA

In [1]:
# Train data.
import pandas as pd
data_train = pd.read_csv("FeatureSelectionOutput.csv")

# split values into inpits and outputs.
values_train = data_train.values
X_train = values_train[:,1:11]
y_train = values_train[:,0]

data_train.shape

(97044, 11)

In [2]:
# Test data.
data_test_full = pd.read_csv("ScaledTestDataSet.csv")

# Create new dataset with features previously selected.
columns_needed = list(data_train.columns)
data_test = data_test_full[columns_needed].copy()

# split values into inpits and outputs.
values_test = data_test.values
X_test = values_test[:,1:11]
y_test = values_test[:,0]

data_test.shape

(40158, 11)

# LOGISTIC REGRESSION

## LR with default hyperparameters

In [3]:
# Initiate the LR model with defualt hyperparameters.
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [4]:
# Fit the model using default hyperparameters.
# K, you don't split into train and validate sets??
lr.fit(X_train, y_train)

C:\Users\K\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [5]:
# Run predictions on TEST set and see the accuracy.
lr.score(X_test,y_test)

0.9626973454853329

In [6]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
lr_predicted = lr.predict(X_test)
print(confusion_matrix(y_test, lr_predicted))

[[18584  1495]
 [    3 20076]]


## LR hyperparameters tuning (Random Search)

In [59]:
from sklearn.model_selection import RandomizedSearchCV

# Create array of values for tuned hyperparameters.
lr_params = {'dual' : [True,False], 
             'C' : [0.1, 0.5, 1.0, 1.5, 2.0, 2.5], 
             'max_iter' : [100, 150, 200, 300, 500, 1000]
             }

In [60]:
# Run random search and initiate the model with tuned parameters.
lr_random = RandomizedSearchCV(estimator=lr, param_distributions=lr_params, cv = 3, n_jobs=-1, random_state = 2019)

import time
start_time = time.time()
lr_random_result = lr_random.fit(X_train, y_train)
finish_time = time.time()

# Summarize results
print("Best: %f using %s" % (lr_random_result.best_score_, lr_random_result.best_params_))
print("Execution time: " + str((finish_time - start_time)))

C:\Users\K\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Best: 0.982297 using {'max_iter': 100, 'dual': False, 'C': 2.0}
Execution time: 6.558957576751709


In [61]:
# Apply best values of hyperparameters to the model.
lr_random = lr_random.best_estimator_

In [62]:
# Train the tuned model on TRAIN set and check the accuracy
lr_random.fit(X_train, y_train)
lr_random.score(X_test,y_test)

C:\Users\K\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9750983614721849

In [63]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
lr_random_predicted = lr_random.predict(X_test)
print(confusion_matrix(y_test, lr_predicted))

[[18584  1495]
 [    3 20076]]


## LR tuning Results

In [64]:
print("LR default hyperparameters test accuracy: ", lr.score(X_test,y_test),', parameters: ', '\n', lr.get_params(),'\n')
print("LR tuned hyperparameters test accuracy: ", lr_random.score(X_test,y_test),', parameters: ', '\n', lr_random.get_params())

LR default hyperparameters test accuracy:  0.9626973454853329 , parameters:  
 {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'warn', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'warn', 'tol': 0.0001, 'verbose': 0, 'warm_start': False} 

LR tuned hyperparameters test accuracy:  0.9750983614721849 , parameters:  
 {'C': 2.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'warn', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'warn', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


# DECISION TREE

## DT with default hyperparameters

In [13]:
# Initiate a DT model using default hyperparameters.
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()

In [14]:
# Train model on train data.
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [15]:
# Check model accuracy on the TEST set.
dt.score(X_test, y_test)

0.9761442302903531

In [16]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, dt.predict(X_test)))

[[19124   955]
 [    3 20076]]


## DT hyperparameters tuning (Random Search)

In [17]:
# Create array of values for tuned hyperparameters.
dt_params = {'max_depth': [None, 0.1, 1, 3, 5, 10], 
             'min_samples_leaf': [0.04, 0.06, 0.08, 1], 
             'max_features': [None, 0.2, 0.4,0.6, 0.8]}

In [18]:
# Run random search.
from sklearn.model_selection import RandomizedSearchCV
dt_random = RandomizedSearchCV(estimator=dt, param_distributions=dt_params, cv = 10, n_jobs=-1, random_state = 2019)

import time
start_time = time.time()
dt_random_result = dt_random.fit(X_train, y_train)
finish_time = time.time()

# Summarize results
print("Best: %f using %s" % (dt_random_result.best_score_, dt_random_result.best_params_))
print("Execution time: " + str((finish_time - start_time)))

Best: 0.987552 using {'min_samples_leaf': 1, 'max_features': 0.8, 'max_depth': None}
Execution time: 4.8630030155181885


In [19]:
# Apply best values of hyperparameters to the model.
dt_random = dt_random.best_estimator_

In [20]:
# Train the tuned model on TRAIN set and check the accuracy
dt_random.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=0.8, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [21]:
# Train the tuned model on TRAIN set and check the accuracy
dt_random.fit(X_train, y_train)
dt_random.score(X_test,y_test)

0.9761442302903531

In [22]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, dt_random.predict(X_test)))

[[19124   955]
 [    3 20076]]


## DT tuning Results

In [23]:
print("DT default hyperparameters test accuracy: ", dt.score(X_test,y_test),', parameters: ', '\n', dt.get_params(),'\n')
print("DT tuned hyperparameters test accuracy: ", dt_random.score(X_test,y_test),', parameters: ', '\n', dt_random.get_params())

DT default hyperparameters test accuracy:  0.9761442302903531 , parameters:  
 {'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': False, 'random_state': None, 'splitter': 'best'} 

DT tuned hyperparameters test accuracy:  0.9761442302903531 , parameters:  
 {'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 0.8, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': False, 'random_state': None, 'splitter': 'best'}


# RANDOM FOREST

## RF with default hyperparameters

In [24]:
# Initiate a RF model using default hyperparameters.
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [25]:
# Train model on train data.
rf.fit(X_train, y_train)

C:\Users\K\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [26]:
# Check model accuracy on the TEST set.
rf.score(X_test, y_test)

0.9761442302903531

In [27]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, rf.predict(X_test)))

[[19124   955]
 [    3 20076]]


## RF hyperparameters tuning (Random Search)

In [28]:
# Define a grid of hyperparameters.
rf_params = { 'n_estimators': [1, 5, 10, 30, 50, 100, 300, 400, 500], 
             'max_depth': [None, 4, 6, 8], 
             'min_samples_leaf': [0.1, 0.2, 0.5, 1], 
             'max_features': ['auto', 'log2', 'sqrt']
            }

In [29]:
# Run random search.
from sklearn.model_selection import RandomizedSearchCV
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=rf_params, cv = 3, n_jobs=-1, random_state = 2019)

import time
start_time = time.time()
rf_random_result = rf_random.fit(X_train, y_train)
finish_time = time.time()

# Summarize results
print("Best: %f using %s" % (rf_random_result.best_score_, rf_random_result.best_params_))
print("Execution time: " + str((finish_time - start_time)))

Best: 0.987583 using {'n_estimators': 1, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None}
Execution time: 47.95043992996216


In [30]:
# Apply best values of hyperparameters to the model.
rf_random = rf_random.best_estimator_

In [31]:
# Train the tuned model on TRAIN set and check the accuracy
rf_random.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## RF tuning Results

In [32]:
print("RF default hyperparameters test accuracy: ", rf.score(X_test,y_test),', parameters: ', '\n', rf.get_params(),'\n')
print("RF tuned hyperparameters test accuracy: ", rf_random.score(X_test,y_test),', parameters: ', '\n', rf_random.get_params())

RF default hyperparameters test accuracy:  0.9761442302903531 , parameters:  
 {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 10, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False} 

RF tuned hyperparameters test accuracy:  0.9761442302903531 , parameters:  
 {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


# SVM (SVC)

## SVC with default hyperparameters

In [34]:
from sklearn import svm
svclassifier = svm.SVC()

In [35]:
svclassifier.fit(X_train, y_train)

C:\Users\K\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [36]:
svclassifier.score(X_test, y_test)

0.9625728372926938

In [37]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, svclassifier.predict(X_test)))

[[18582  1497]
 [    6 20073]]


 ## SVC hyperparameters tuning (Random Search)

In [38]:
# Define a grid of hyperparameters.
svc_params = { 'C': [0.1, 0.5, 1, 3, 5], 
             'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
             'degree': [2, 3, 4], 
              'gamma': [0.01, 0.1, 1, 10]
            }

In [39]:
# Run random search.
from sklearn.model_selection import RandomizedSearchCV
svc_random = RandomizedSearchCV(estimator=svclassifier, n_iter=3, param_distributions=svc_params, cv = 3, n_jobs=-1, 
                                random_state = 2019)

import time
start_time = time.time()
svc_random_result = svc_random.fit(X_train, y_train)
finish_time = time.time()

# Summarize results
print("Best: %f using %s" % (svc_random_result.best_score_, svc_random_result.best_params_))
print("Execution time: " + str((finish_time - start_time)))

Best: 0.986810 using {'kernel': 'poly', 'gamma': 10, 'degree': 3, 'C': 3}
Execution time: 114.23359298706055


In [41]:
# Apply best values of hyperparameters to the model.
svc_random = svc_random.best_estimator_

In [43]:
# Train the tuned model on TRAIN set and check the accuracy
svc_random.fit(X_train, y_train)
svc_random.score(X_test,y_test)

0.9734797549678769

In [44]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, svc_random.predict(X_test)))

[[19017  1062]
 [    3 20076]]


## SVC tuning Results

In [45]:
print("SVC default hyperparameters test accuracy: ", svclassifier.score(X_test,y_test), 
      ', parameters: ', '\n', svclassifier.get_params(),'\n')
print("SVC tuned hyperparameters test accuracy: ", svc_random.score(X_test,y_test), 
      ', parameters: ', '\n', svc_random.get_params())

SVC default hyperparameters test accuracy:  0.9625728372926938 , parameters:  
 {'C': 1.0, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'auto_deprecated', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False} 

SVC tuned hyperparameters test accuracy:  0.9734797549678769 , parameters:  
 {'C': 3, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 10, 'kernel': 'poly', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}


# Compare Algorithms Performance

In [65]:
print("LR tuned hyperparameters test accuracy: ", lr_random.score(X_test,y_test))
print("DT tuned hyperparameters test accuracy: ", dt_random.score(X_test,y_test))
print("RF tuned hyperparameters test accuracy: ", rf_random.score(X_test,y_test))
print("SVC tuned hyperparameters test accuracy: ", svc_random.score(X_test,y_test))

LR tuned hyperparameters test accuracy:  0.9750983614721849
DT tuned hyperparameters test accuracy:  0.9761442302903531
RF tuned hyperparameters test accuracy:  0.9761442302903531
SVC tuned hyperparameters test accuracy:  0.9734797549678769
